# Pose Detection with AlphaPhose v0.3.0

This notebook uses an open source project [MVIG-SJTU/AlphaPose](https://github.com/MVIG-SJTU/AlphaPose) to detect/track multi person poses on a given youtube video.



## Install AlphaPose v0.3.0

In [2]:
!pip install -U torch==1.4 torchvision==0.5 -f https://download.pytorch.org/whl/cu101/torch_stable.html
import os
from os.path import exists, join, basename, splitext

git_repo_url = 'https://github.com/MVIG-SJTU/AlphaPose.git'
project_name = splitext(basename(git_repo_url))[0]
if not exists(project_name):
  # clone and install dependencies
  !git clone -q {git_repo_url}
  !pip install -q youtube-dl cython gdown
  !pip install -q -U PyYAML
  !apt-get install -y -q libyaml-dev
  !cd {project_name} && git checkout 7be9809 && python setup.py build develop --user
  
import sys
sys.path.append(project_name)
import time
import matplotlib
import matplotlib.pylab as plt
plt.rcParams["axes.grid"] = False

from IPython.display import YouTubeVideo

Looking in links: https://download.pytorch.org/whl/cu101/torch_stable.html
Reading package lists...
Building dependency tree...
Reading state information...
libyaml-dev is already the newest version (0.1.7-2ubuntu3).
0 upgraded, 0 newly installed, 0 to remove and 37 not upgraded.
Note: checking out '7be9809'.

You are in 'detached HEAD' state. You can look around, make experimental
changes and commit them, and you can discard any commits you make in this
state without impacting any branches by performing another checkout.

If you want to create a new branch to retain commits you create, you may
do so (now or later) by using -b with the checkout command again. Example:

  git checkout -b <new-branch-name>

HEAD is now at 7be9809 update setup.py
running build
running build_py
creating build
creating build/lib.linux-x86_64-3.7
creating build/lib.linux-x86_64-3.7/alphapose
copying alphapose/__init__.py -> build/lib.linux-x86_64-3.7/alphapose
copying alphapose/opt.py -> build/lib.linux-x86_

## Download pretrained models

In [3]:
yolo_pretrained_model_path = join(project_name, 'detector/yolo/data/yolov3-spp.weights')
if not exists(yolo_pretrained_model_path):
  # download the YOLO weights
  !mkdir -p {project_name}/detector/yolo/data
  !gdown -O {yolo_pretrained_model_path} https://drive.google.com/uc?id=1D47msNOOiJKvPOXlnpyzdKA3k6E97NTC

#tracker_pretrained_model_path = join(project_name, 'detector/tracker/data/jde.1088x608.uncertainty.pt')
#if not exists(tracker_pretrained_model_path):
#  # tracker weights
#  !mkdir -p {project_name}/detector/tracker/data
#  !gdown -O {tracker_pretrained_model_path} https://drive.google.com/uc?id=1nlnuYfGNuHWZztQHXwVZSL_FvfE551pA

# ResNet152 backbone 73.3 AP
pretrained_model_path = join(project_name, 'pretrained_models/fast_421_res152_256x192.pth')
pretrained_model_config_path = join(project_name, 'configs/coco/resnet/256x192_res152_lr1e-3_1x-duc.yaml')
if not exists(pretrained_model_path):
  # download the pretrained model
  !gdown -O {pretrained_model_path} https://drive.google.com/uc?id=1kfyedqyn8exjbbNmYq8XGd2EooQjPtF9

Downloading...
From: https://drive.google.com/uc?id=1D47msNOOiJKvPOXlnpyzdKA3k6E97NTC
To: /content/AlphaPose/AlphaPose/detector/yolo/data/yolov3-spp.weights
100% 252M/252M [00:01<00:00, 238MB/s]
Downloading...
From: https://drive.google.com/uc?id=1kfyedqyn8exjbbNmYq8XGd2EooQjPtF9
To: /content/AlphaPose/AlphaPose/pretrained_models/fast_421_res152_256x192.pth
100% 334M/334M [00:01<00:00, 273MB/s]


## Detect poses on a test video

We are going to detect poses on the following youtube video:

Download the above youtube video, cut the first 5 seconds and do the pose detection on that 5 seconds:

In [ ]:
!rm -df youtube.mp4
# download the youtube with the given ID
!youtube-dl -f 'bestvideo[ext=mp4]' --output "youtube.%(ext)s" https://www.youtube.com/watch?v={YOUTUBE_ID}
# cut the first 5 seconds
!ffmpeg -y -loglevel info -i youtube.mp4 -t 5 video.avi
# run AlpaPose on these 5 seconds video
!rm -rf AlphaPose_video.avi
!cd {project_name} && python3 scripts/demo_inference.py --sp --video ../video.avi --outdir ../ --save_video --checkpoint ../{pretrained_model_path} --cfg ../{pretrained_model_config_path}
# convert the result into MP4
!ffmpeg -y -loglevel info -i AlphaPose_video.avi -vcodec libx264 AlphaPose_video.mp4

Finally, visualize the result:

In [ ]:
import glob
import subprocess

video_file_names = glob.glob("/content/drive/MyDrive/FastCode_AI_HumanPose_PoC/*.mp4")
frame_rate = 30

# remember to change into the parent directory at the end 
for video_file in video_file_names:
  print("\n")
  print("now processing... ->", os.path.basename(video_file))
  
  video_name = os.path.basename(video_file).split(".")[0]
  processed_folder = os.path.join("/content/", str(video_name))
  # frame_folder_path = os.path.dirname(frame_folder_path)
  if not os.path.exists(processed_folder):
    print("creating directory...", processed_folder)
    os.makedirs(processed_folder)
  else:
    print(processed_folder, "already exists ; proceeding...")

  !python3 scripts/demo_inference.py --sp --video {video_file} --outdir {processed_folder} --save_video --checkpoint ../{pretrained_model_path} --cfg ../{pretrained_model_config_path}
  # break





now processing... -> HighKnees.mp4
/content/HighKnees already exists ; proceeding...
Load SE Resnet...
Loading YOLO model..
Network successfully loaded
Loading pose model from ../AlphaPose/pretrained_models/fast_421_res152_256x192.pth...
Could not find encoder for codec id 27: Encoder not found
  0% 0/2424 [00:00<?, ?it/s]Try to use other video encoders...
100% 2424/2424 [03:08<00:00, 12.83it/s]
===========================> Finish Model Running.
===========================> Rendering remaining images in the queue...
===========================> If this step takes too long, you can enable the --vis_fast flag to use fast rendering (real-time).
===========================> Rendering remaining 187 images in the queue...
===========================> Rendering remaining 139 images in the queue...
===========================> Rendering remaining 91 images in the queue...
===========================> Rendering remaining 44 images in the queue...
===========================> Rendering remaini